## Dynamic Discrete Choice Model - Rust and BBL
### Xiang Liu
#### Department of Economics, University of Minnesota

This project replicates the Nested Fixed Point Algorithm used by John Rust in his 1987 seminal paper "Optimal Replacement of GMC Bus Engines: An Empirical Model of Harold Zurcher”. Then, I estimate Rust's results using Bajari, Benkard, and Levin (henthforce BBL, 2007) methods. 

I gained a lot of help from Chonghao Hao for these exercises. Credit to Hao. 

The codes here maybe not efficient, the Python version with JIT and GPU is coming up soon (hopefully). 

In [1]:
using CSV, DataFrames, Optim, Random, LinearAlgebra, ForwardDiff, Statistics, Random,Base.Threads,JLD2,Plots
using GLM,Distributions
using Base.Threads: @threads
using LaTeXStrings,Latexify, Printf

## Functions needed

#### Nested Fixed Point Algorithm in Rust (1987)

In [2]:
mutable struct Zurcher
    # containing default parameters
        n::Int
        max::Int
        beta::Float64
        grid::Vector{Int}
    
        function Zurcher(;n = 90, max = 450, beta = 0.9999)
            grid = collect(0:(n-1))
            new(n, max, beta, grid)
        end
    end
    
    function calculate_proportions(data)
    # calculate transition prob from data
        unique_values = unique(data.dx1)
        p = Vector{Float64}(undef, length(unique_values) - 1)
        
        for (i, val) in enumerate(unique_values[1:end-1])
            p[i] = sum(data.dx1 .== val) / length(data.dx1)
        end
    
        return p
    end
    
    function transition_probs(n, p)
    # calculate transition prob matrix
        trpr = zeros(n, n)
        probs = vcat(p, 1 - sum(p))
        for (i, p) in enumerate(probs)
            trpr += diagm(i-1 => fill(p, n-i+1))
        end
        trpr[:, end] .= 1 .- sum(trpr[:, 1:end-1], dims=2)
        return trpr
    end
    
    
    struct Dta
    # contain data for estimation
        d::Vector{Float64}          # replace dummy, d_t
        x::Vector{Float64}          # Odometer, x_t
        dx1::Vector{Float64}        # Monthly mileage, dx1_t
    end
    
    function readbusdata(mp, bustypes)
        data = CSV.read("stata_rust_data.csv", DataFrame);
        # Bus id, bustype, d1 Lagged replacement dummy, d Replacement dummy, x Odometer, dx1 Monthly mileage
        df = DataFrame(id = data[:, 1], bustype = data[:, 2], d1 = data[:, 5], x = data[:, 7])
        # Select buses
        if bustypes !== nothing
            df = filter(row -> row[2] in bustypes, df)
        end
    
        # Discretize odometer data into 1, 2, ..., n
        df.x = ceil.(df.x .* mp.n / (mp.max * 1000))
    
        processed_groups = []
    
        grouped_df = groupby(df, :id)
        for group in grouped_df
            # Calculate d for each group
            d = [group.d1[2:end]; 0]  # Shift and append 0
            group.d = d 
        
            # Calculate dx1
            dx1 = group.x - [0; group.x[1:end-1]]
            dx1 = dx1 .* (1 .- group.d1) .+ group.x .* group.d1
            group.dx1 = dx1
        
            # Drop the first row of each group by selecting the rest of the rows
            push!(processed_groups, group[2:end, :])
        end
        
        # Combine the processed groups back into a single DataFrame
        final_df = vcat(processed_groups...)
        final_df = dropmissing(final_df, [:d, :x, :dx1])
        return final_df
    end
    
    # Bellman equation
    function bellman(mp::Zurcher, ev0, c, RC, trpr)
        x = mp.grid
        cost = -0.001 * x * c                       # cost function
        vx0 = cost + mp.beta * ev0                  # value of keeping
        vx1 = -RC + cost[1] + mp.beta * ev0[1]      # value of replacing
    
        maxV = max.(vx0, vx1)
        ev1 = trpr * (maxV + log.(exp.(vx0 .- maxV) + exp.(vx1 .- maxV)))
        pr1 = 1 ./ (exp.(vx1 .- vx0) .+ 1)
        return ev1, pr1
    end
    
    function solve_vfi(c, RC, trpr, mp::Zurcher, tol = 1e-6, maxiter = 1e10)
    # value iterated function
        ev0 = zeros(mp.n)
        ev1 = copy(ev0)  
        pr1 = copy(ev0)  
    
        for i in 1:maxiter
            ev1, pr1 = bellman(mp, ev0, c, RC, trpr)
            err = maximum(abs.(ev0 - ev1))
    
            if err < tol
                break
            end
            ev0 = ev1
        end
    
        if maximum(abs.(ev0 - ev1)) >= tol
            error("Failed to converge in $maxiter iterations")
        end
    
        return ev1, pr1
    end
    
    
    
    function ll(data, mp, theta::Vector,pp=0)
    # maximum likelihood function
        if length(theta) >= 2
            RC = theta[1]
            c = theta[2]
            if length(theta) > 2
                p = theta[3:end]
            else
                p = pp
                trpr = transition_probs(mp.n, p)
            end
        end
    
        # p = abs.(p) # used for unconstrained optimization
        if any(x -> x < 0, p)
            return Inf
        end
    
        if sum(p) > 1.0
            return Inf
        end
        
        if length(theta) > 2
            trpr = transition_probs(mp.n, p)
        end
    
        # Solve model
        ev0, pr = solve_vfi(c, RC, trpr,mp)
    
        # Evaluate likelihood function
    
        # 1. log likelihood regarding replacement choice
        lp = [ pr[convert(Int, i)] for i in data.x ] # map the probabilities to the corresponding odometer x_t
        logl = log.(lp .+ (1 .- 2 .* lp) .* data.d)
    
        # 2. add on log like for mileage process
        if length(theta) > 2
            p = vcat(p, 1 - sum(p)) # can be negative in unconstrained optimization
            n_p = length(p) - 1
            pr_dx = [ p[1 + convert(Int, i)] for i in data.dx1 ] 
            logl .+= log.(pr_dx)   # Monthly mileage, dx1_t
        else
            n_p = 0
        end
    
        # Objective function (negative mean log likelihood)
        f = sum(-logl)
        return f,ev0
    end

ll (generic function with 2 methods)

## Functions needed

#### BBL

In [32]:
# BBL Estimator

# I vectorize perturbed policies and states to speed up the computation
# The commented code is calculation for each state before vectorization 
# Did not calculate se 
###################################################################################################

function get_ccp(data,mp)
    # estimate the conditional choice probability by a logistic regression
    # P(d=1|mileage)
    # return prob of replacement
        X = hcat(ones(length(data.x)), data.x)  # Adding a column of ones for the intercept
        y = data.d
    
        # Fitting the model
        my_logit = glm(X, y, Binomial(), LogitLink())
        ccp = predict(my_logit,hcat(ones(length(mp.grid)), mp.grid))
        return ccp
    end
    
    function optimal_policy(ϵ_k, ϵ_r, optimal_policy_cutoff)
        result = (ϵ_r .- ϵ_k) .> optimal_policy_cutoff
        int_result = Int.(result)
        return int_result
    end
    
    function gen_path(β::Float64, optimal_cutoff::Matrix{Float64}, p::Vector{Float64}, T::Int64)
        # β discount factor 
        # optimal_cutoff: optimal cutoff for each state 
        # p: Δ mileage transition probability
        n, m = size(optimal_cutoff)
        s_0 = repeat(1:n, 1, m) # Initial state for different optimal_cutff
        sum_w1 = zeros(n,m)
        sum_w2 = copy(sum_w1)
        sum_w3 = copy(sum_w1)
        dist_ϵ = Gumbel(-0.5772, 1)
        dis_Δ = Categorical(p)
        # Sum T periods
        for t in 1:T
            # Type I extreme value distribution with mean 0
            ϵ_0 = rand(dist_ϵ, n) # can also generate n*m matrix
            ϵ_1 = rand(dist_ϵ, n)
            # Probability of replacement
            cutoff_s0 = hcat([optimal_cutoff[s_0[:, j], j] for j in 1:size(optimal_cutoff, 2)]...)
            a_0 = optimal_policy(ϵ_0, ϵ_1, cutoff_s0) # Optimal action conditional on s_0
    
            #  Sum of value function
            # sum_V += a_0 * (-RC - 0.001 * θ_1 + ϵ_1) * β^t + 
            #          (1 - a_0) * (-0.001 * θ_1 * s_0 + ϵ_0) * β^t 
            sum_w1 .+= a_0 .* β^(t-1)
            sum_w2 .+= 0.001 .* (1 .- a_0) .*s_0 .* β^(t-1)
            sum_w3 .+= ifelse.(a_0 .== 1, ϵ_1, ϵ_0).* β^(t-1)
            # Generate s_1
            # generate Δ meliage
            ΔX = rand(dis_Δ, n)
    
            # If replace Δx; if not Δx + s_0
            s_1 = ifelse.(a_0 .== 1, ΔX, ΔX .- 1 .+ s_0)
    
            # Ensuring s_1 does not exceed max_mile
            s_1 = min.(s_1, n)
    
            # Update s_0, pr_s
            s_0 = s_1
            
        end
        return sum_w1,sum_w2, sum_w3
    end
    
    function gen_perturbs(ccp::Vector{Float64}, N::Int,dis = Normal(0, 1))
        # Initialize an array to hold the perturbations
        perturbations = Array{Float64,2}(undef, length(ccp), N)
    
        # Generate each perturbation
        for i in 1:N
            perturbations[:, i] = ccp .+ rand(dis,length(ccp))
        end
    
        return perturbations
    end
    
    function mean_matrixes(res::Vector{Matrix{Float64}})
        # Caluculate mean of matrixes
        # each col of matrix represents a perturbation, each row represents a state
        stacked_matrices = cat(res..., dims=3)
        average_matrix = mean(stacked_matrices, dims=3)
        average_matrix = dropdims(average_matrix, dims=3)
        return average_matrix
    end 
    
    function get_Ev(β, optimal_cutoffs, p, tol, iterations)
        w1s = Array{Matrix{Float64}, 1}(undef, iterations) #ensure thread safe
        w2s = Array{Matrix{Float64}, 1}(undef, iterations)
        w3s = Array{Matrix{Float64}, 1}(undef, iterations)
        
        @threads for i in 1:iterations
            sum_V = gen_path(β, optimal_cutoffs, p, tol)
            w1s[i] = sum_V[1]
            w2s[i] = sum_V[2]
            w3s[i] = sum_V[3]
        end
        Ew1 = mean_matrixes(w1s)
        Ew2 = mean_matrixes(w2s)
        Ew3 = mean_matrixes(w3s)
        return [Ew1, Ew2, Ew3]
    end
    
    function value_func(sum_w1,sum_w2,sum_w3,R, θ)
        # sum_V += a_0 * (-RC - 0.001 * θ_1 + ϵ_1) * β^t + (1 - a_0) * (-0.001 * θ_1 * s_0 + ϵ_0) * β^t
        sum_V =  -R .* sum_w1 .- θ .* sum_w2 .+ sum_w3
        return sum_V
    end
    
    function obj_func(res,J,pars)
        # res get from func get_Ev() has 3 matrix sum_w1,sum_w2,sum_w3
        # J is the number of perturbations
        R = pars[1]
        θ = pars[2]
        obj = 0
        sum_w1,sum_w2,sum_w3 = res
        for j in 2:J
            V₀ = value_func(sum_w1[:,1],sum_w2[:,1],sum_w3[:,1],R, θ)
            Vⱼ = value_func(sum_w1[:,j],sum_w2[:,j],sum_w3[:,j],R, θ)
            g = min.(V₀-Vⱼ,0)
            obj += sum(g.^2)
        end
        return obj
    end

obj_func (generic function with 1 method)

In [33]:
mp = Zurcher();

In [34]:
function MLE_rust(mp, data)
    Q(e) = ll(data, mp, e)[1] # define maximum likelihood function preload data and only output the likelihood
    theta = [0,0,0.1,0.1] # initial guess
    theta_h = optimize(Q,theta) # optimize the function
    θ1 = Optim.minimizer(theta_h)
    return θ1
end

MLE_rust (generic function with 1 method)

In [35]:
# function to process each group
function process_group(mp, group)
    df = readbusdata(mp,group)
    data = Dta(df.d,df.x,df.dx1)

    Q(e) = ll(data, mp, e)[1]  # define Q again to calculate Hessian matrix
    θ1 = MLE_rust(mp, data) # optimal results
    likelihood = -ll(data, mp, θ1)[1]  
    
    # use the inverse of Hessian matrix to calculate se 
    H = ForwardDiff.hessian(Q, θ1) 
    se = sqrt.(diag(inv(H)) / length(θ1))
    N = size(df, 1)
    return (θ1, se, likelihood, N) # estimators for table
end

process_group (generic function with 1 method)

## Exercise 1

#### Replicate Rust Paper table IX for beta =0.999 and 0 

In [40]:
# Test different types of bus same as rust paper Table IX
# β = 0.999 is default 
groups = [[1,2,3], [4], [1,2,3,4]]

# preset vector for multithreading
all_data = Vector{Tuple}(undef, length(groups))
@threads for i in 1:length(groups)
    group = groups[i]
    print(group)
    # for each group get θ and se and likelihood
    θ1, se, likelihood, N = process_group(mp, group)
    all_data[i] = (θ1, se, likelihood, N)
end


[1, 2, 3][4][1, 2, 3, 4]

In [41]:
# same test as above with β = 0
groups = [[1,2,3], [4], [1,2,3,4]]
all_data_0 = Vector{Tuple}(undef, length(groups))
mp = Zurcher()
mp.beta = 0
@threads for i in 1:length(groups)
    group = groups[i]
    print(group)
    θ1, se, likelihood, N = process_group(mp, group)
    all_data_0[i] = (θ1, se, likelihood, N)
end


[1, 2, 3][4][1, 2, 3, 4]

In [42]:
# # Output latex (uncomment the whole cell if you want to do this)
# function generate_latex_table_for_dataset(dataset, subtitle)
#     groups = ["Groups 1, 2, 3", "Group 4", "Groups 1, 2, 3, 4"]
#     row_titles = ["RC", "\$\\theta_{11}\$", "\$\\theta_{30}\$", "\$\\theta_{31}\$", "Log-likelihood", "N"]

#     # Start the LaTeX table for this dataset
#     latex_table = """
#     \\begin{center}
#     \\textbf{$subtitle}
#     \\end{center}
#     \\begin{tabular}{|c|$(repeat("c|", length(groups)))}
#     \\hline
#     Parameter & $(join(groups, " & ")) \\\\ \\hline
#     """

#     # Iterate over each row (parameter)
#     for (row_idx, row_title) in enumerate(row_titles)
#         latex_table *= row_title

#         # Add data for each group
#         for (group_idx, data) in enumerate(dataset)
#             θ1, se, likelihood, N = data
#             if row_idx <= length(θ1)
#                 # Format θ1 and se values
#                 value = @sprintf("%.4f (%.4f)", θ1[row_idx], se[row_idx])
#             elseif row_idx == length(row_titles) - 1
#                 # Format likelihood
#                 value = @sprintf("%.4f", likelihood)
#             else
#                 # Format N
#                 value = @sprintf("%.4f", N)
#             end

#             latex_table *= " & $value"
#         end

#         latex_table *= " \\\\ \\hline\n"
#     end

#     # Close the table for this dataset
#     latex_table *= "\\end{tabular}\n"

#     return latex_table
# end

# # Combine tables for all_data and all_data_0
# latex_table_all_data = generate_latex_table_for_dataset(all_data, "Beta is 0.9999")
# latex_table_all_data_0 = generate_latex_table_for_dataset(all_data_0, "Beta is 0")

# # Full LaTeX table
# full_latex_table = """
# \\begin{table}[ht]
# \\centering
# \\caption{Rust estimators for different groups}
# $latex_table_all_data
# \\vspace{1cm} % Space between tables
# $latex_table_all_data_0
# \\label{your_table_label}
# \\end{table}
# """

# open("rust_combined_tables.txt", "w") do file
#     write(file, full_latex_table)
# end


## Exercise 2

#### Test different β values for all bus types 1,2,3,4

In [43]:
group = [1, 2, 3, 4]
β_values = [0.1, 0.3,0.5, 0.7, 0.9]

# Preallocate the results array with the proper size
betas_data = Array{Tuple}(undef, length(β_values))

# Use a thread-safe operation
@threads for i in 1:length(β_values)
    local_mp = deepcopy(mp)  # Create a local copy
    local_mp.beta = β_values[i]
    θ1, se, likelihood, N = process_group(local_mp, group)
    betas_data[i] = (θ1, se, likelihood, N)
end
betas_data

5-element Vector{Tuple}:
 ([7.311520378934131, 63.4417912413077, 0.34882285806613333, 0.6394066051276539], [0.18576751359476773, 3.4618861938199155, 0.002638659859291193, 0.00265845204122969], -6061.6086520304425, 8156)
 ([7.328900199407271, 49.77852951688872, 0.3488224776588006, 0.6394069754452629], [0.18738465751583555, 2.7317181444014693, 0.0026386585420751104, 0.0026584508669865287], -6061.515177205987, 8156)
 ([7.360898054944011, 36.130264226637365, 0.3488216043653711, 0.6394077045215878], [0.1903875712872247, 2.0029636919935143, 0.0026386555714725717, 0.0026584486065551905], -6061.3455936791925, 8156)
 ([7.438389598625533, 22.520464191922056, 0.34881883990090085, 0.6394104405428604], [0.1977559067829231, 1.2776191678956994, 0.0026386461058494916, 0.0026584400260152886], -6060.949358691211, 8156)
 ([7.824363293560287, 9.047816625436504, 0.3488101975559138, 0.6394184243689262], [0.23594230661459548, 0.5640727305138774, 0.0026386016204377892, 0.00265840217780538], -6059.264101943403

In [44]:
# # Output Latex (uncomment the whole cell if you want to do this)
# # Row titles and β values
# row_titles = ["RC", "\$\\theta_{11}\$", "\$\\theta_{30}\$", "\$\\theta_{31}\$", "Log-likelihood", "N"]

# # Start the LaTeX table
# latex_table = """
# \\begin{table}[ht]
# \\centering
# \\begin{tabular}{|c|$(repeat("c|", length(β_values)))}
# \\hline
# Parameter & $(join(["\\(\\beta\\)=$(β)" for β in β_values], " & ")) \\\\ \\hline
# """

# # Iterate over each row (parameter)
# for (row_idx, row_title) in enumerate(row_titles)
#     latex_table *= row_title
#     for (β_idx, data) in enumerate(betas_data)
#         θ1, se, likelihood, N = data
#         if row_idx <= length(θ1)
#             value = @sprintf("%.4f (%.4f)", θ1[row_idx], se[row_idx])
#         elseif row_idx == length(row_titles) - 1
#             value = @sprintf("%.4f", likelihood)
#         else
#             value = @sprintf("%.4f", N)
#         end
#         latex_table *= " & $value"
#     end
#     latex_table *= " \\\\ \\hline\n"
# end

# # Close the LaTeX table
# latex_table *= "\\end{tabular}\n\\caption{Rust_betas}\n\\label{table:rust_betas}\n\\end{table}"

# # Write to file
# open("rust_betas.tex", "w") do file
#     write(file, latex_table)
# end


## Exercise 3


#### Quadruple Grid Size
Can't use NFXP to estimate transition matrix. When grids is large, it will have 11 states and yields abnormal result. 

Instead, I estimate mileage transition matrix from data

In [45]:
function large_grid_MLE(mp)
    group = [1,2,3,4]
    df = readbusdata(mp,group)
    data = Dta(df.d,df.x,df.dx1)
    pp = calculate_proportions(data)
    Q(e) = ll(data, mp, e,pp)[1]
    # can't use NXP to estimate transition matrix when n is large, it yields abornal result
    # theta = fill(0.0, length(unique(data.dx1))+1) 
    theta = [0.0, 0.0]
    theta_h = optimize(Q,theta)
    θ1 = Optim.minimizer(theta_h) 
    likelihood = -ll(data, mp, θ1,pp)[1]  
    H = ForwardDiff.hessian(Q, θ1) 
    se = sqrt.(diag(inv(H)) / length(θ1))
    N = size(df, 1)
    return (θ1, se, likelihood, N)
end

large_grid_MLE (generic function with 1 method)

In [46]:
mp = Zurcher(n = 360, max = 450, beta = 0)
res0=large_grid_MLE(mp)

([7.323396619936107, 17.524120991742485], [0.2633492884124831, 1.3519051055337161], -306.8520665119867, 8156)

In [47]:
mp = Zurcher(n = 360, max = 450, beta = 0.9999)
res1=large_grid_MLE(mp)

InterruptException: InterruptException:

In [48]:
# # Output latex (uncomment the whole cell if you want to do this)
# betas_data = [res0,res1]
# β_values = [0.0, 0.9999]
# # Row titles
# row_titles = ["RC", "\$\\theta_{11}\$", "Log-likelihood", "N"]

# # Start the LaTeX table
# latex_table = """
# \\begin{table}[ht]
# \\centering
# \\begin{tabular}{|c|$(repeat("c|", length(β_values)))}
# \\hline
# Parameter & $(join(["\\(\\beta\\)=$(β)" for β in β_values], " & ")) \\\\ \\hline
# """

# # Iterate over each row (parameter)
# for (row_idx, row_title) in enumerate(row_titles)
#     latex_table *= row_title

#     # Add data for each β value
#     for (β_idx, data) in enumerate(betas_data)
#         θ1, se, likelihood, N = data
#         if row_idx <= length(θ1)
#             # Format θ1 and se values
#             value = @sprintf("%.4f (%.4f)", θ1[row_idx], se[row_idx])
#         elseif row_idx == length(row_titles) - 1
#             # Format likelihood
#             value = @sprintf("%.4f", likelihood)
#         else
#             # Format N
#             value = @sprintf("%.4f", N)
#         end

#         latex_table *= " & $value"
#     end

#     latex_table *= " \\\\ \\hline\n"
# end

# # Close the LaTeX table
# latex_table *= "\\end{tabular}\n\\caption{Rust_betas}\n\\label{your_table_label}\n\\end{table}"
# open("large_grid.txt", "w") do file
#     write(file, latex_table)
# end

# BBL Method

In [49]:
mp = Zurcher()
df = readbusdata(mp,[1,2,3,4])
data = Dta(df.d,df.x,df.dx1)
# Estimate mileage transition matrix
p = calculate_proportions(data)
push!(p, 1 - sum(p))

3-element Vector{Float64}:
 0.6394065718489456
 0.3488229524276606
 0.011770475723393847

In [50]:
# Parameters
function simulate_perturb(n_I,tol;β = mp.beta,iterations = 1000,mp=mp,data=data)
    tol =100 #making calculation faster
    ccp = get_ccp(data, mp)
    cutoff = log.(1 .- ccp) - log.(ccp)
    perturbations = gen_perturbs(cutoff,n_I,Normal(0,0.5))
    optimal_cutoffs = hcat(cutoff,perturbations)
    res = get_Ev(β, optimal_cutoffs, p, tol, iterations)
    return res
end

simulate_perturb (generic function with 1 method)

In [51]:
### Plot for simulated value function when T and S are large enough
tol= 0.99
one_cutoff = reshape(cutoff, length(cutoff), 1)
res = get_Ev(β,one_cutoff, p, tol, 10000)
sum_w1,sum_w2, sum_w3 = res
R =9.7558
θ = 2.6275
dt = value_func(sum_w1[:,1],sum_w2[:,1],sum_w3[:,1],R, θ)
plot(dt, title = "Simulated Value func Plot", xlabel = "s_0", ylabel = "Value", legend = false)
# savefig("simulated_value_function.png")

UndefVarError: UndefVarError: cutoff not defined

In [52]:
function get_theta(res;J=n_I)
    theta = [10.0,1.0]
    Q(pars) = obj_func(res,J,pars)
    theta_hat = optimize(Q,theta)
    θ1 = Optim.minimizer(theta_hat)
    return θ1
end

get_theta (generic function with 1 method)

In [53]:
# Generate latex table for estimators under different β
β_values = [0.1, 0.3, 0.7, 0.9]
results_df = DataFrame()

# Loop through each value of β
for b in β_values
    res = simulate_perturb(n_I, T, β=b)
    theta_res = get_theta(res)
    RC, θ1 = theta_res
    push!(results_df, (β = b, RC = RC, θ1 = θ1))
end

In [ ]:
### β = 0.9999
n_I = 200
T = 100
res = simulate_perturb(n_I,T)
theta0 = get_theta(res)
push!(results_df, (β = mp.beta, RC = theta0[1], θ1 = theta0[2]))

In [ ]:
# β = 0
# it equals to only 1 time period
ccp = get_ccp(data, mp)
cutoff = log.(1 .- ccp) - log.(ccp)
perturbations = gen_perturbs(cutoff,200,Normal(0,0.5))
optimal_cutoffs = hcat(cutoff,perturbations)
@time res = get_Ev(mp.beta, optimal_cutoffs, p, 1, 1000)
theta0 = get_theta(res,J=201)
push!(results_df, (β = 0, RC = theta0[1], θ1 = theta0[2]))

In [ ]:
# # Output Latex (uncomment the whole cell if you want to do this)
# # Rename the columns for LaTeX formatting
# rename!(results_df, Symbol("\\beta") => LaTeXString("\\beta"), :RC => LaTeXString("RC"), Symbol("\\theta_1") => LaTeXString("\\theta_1"))
# foreach(col -> (eltype(results_df[:, col]) <: Number) && 
#     (results_df[:, col] = round.(results_df[:, col], digits=4)), 
#     names(results_df))

# open("BBL_betas.txt", "w") do file
#     title = "BBL estimators for different \$\\beta\$"
#     header = join(["\\(\\beta\\)", "RC", "\\(\\theta_{1}\\)"], " & ") * " \\\\\\hline\n"

#     # Manually construct table rows
#     table_rows = join([join(row, " & ") * " \\\\\\hline" for row in eachrow(results_df)], "\n")

#     # Full LaTeX table
#     full_latex = """
#     \\begin{table}[h!]
#     \\centering
#     \\caption{$(title)}
#     \\begin{tabular}{|c|c|c|}
#     \\hline
#     $header
#     $table_rows
#     \\end{tabular}
#     \\end{table}
#     """
    
#     write(file, full_latex)
# end


#### Quadruple the Grid Size (BBL)

In [ ]:
n_I = 200 # number of perturbations
T = 1 # β=0 equal to only summing 1 period 
mp = Zurcher(n = 360)
df = readbusdata(mp,[1,2,3,4])
data = Dta(df.d,df.x,df.dx1)
# Estimate mileage transition matrix
p = calculate_proportions(data)
push!(p, 1 - sum(p))

ccp = get_ccp(data, mp)
cutoff = log.(1 .- ccp) - log.(ccp)
perturbations = gen_perturbs(cutoff,n_I,Normal(0,0.5))
optimal_cutoffs = hcat(cutoff,perturbations)
@time res = get_Ev(mp.beta, optimal_cutoffs, p, T, 1000)

theta0 = get_theta(res,J=n_I+1)
res0 = (β = 0, RC = theta0[1], θ1 = theta0[2])

In [ ]:
n_I = 200
T = 100 # summing 100 periods to save time, should summing large enough periods to make β^T close to 0
mp = Zurcher(n = 360)
df = readbusdata(mp,[1,2,3,4])
data = Dta(df.d,df.x,df.dx1)
# Estimate mileage transition matrix
p = calculate_proportions(data)
push!(p, 1 - sum(p))

ccp = get_ccp(data, mp)
cutoff = log.(1 .- ccp) - log.(ccp)
perturbations = gen_perturbs(cutoff,200,Normal(0,0.5))
optimal_cutoffs = hcat(cutoff,perturbations)
@time res = get_Ev(mp.beta, optimal_cutoffs, p, T, 1000)
theta0 = get_theta(res,J=n_I+1)
res1 = (β = 0.999, RC = theta0[1], θ1 = theta0[2])
# @time gen_path(mp.beta, optimal_cutoffs, p, T)

In [ ]:
# # output latex (uncomment the whole cell if you want to do this)
# betas_data = [(res0.RC, res0.θ1),(res1.RC, res1.θ1)]
# β_values = [0.0, 0.9999]
# # Row titles
# row_titles = ["RC", "\$\\theta_{1}\$"]

# # Start the LaTeX table
# latex_table = """
# \\begin{table}[ht]
# \\centering
# \\begin{tabular}{|c|$(repeat("c|", length(β_values)))}
# \\hline
# Parameter & $(join(["\\(\\beta\\)=$(β)" for β in β_values], " & ")) \\\\ \\hline
# """

# # Iterate over each row (parameter)
# for (row_idx, row_title) in enumerate(row_titles)
#     latex_table *= row_title
#     for data in betas_data
#         value = "NA" # Default value
#         if row_idx <= length(data)
#             value = @sprintf("%.4f", data[row_idx])
#         end
#         latex_table *= " & $value"
#     end
#     latex_table *= " \\\\ \\hline\n"
# end

# # Close the LaTeX table
# latex_table *= "\\end{tabular}\n\\caption{Rust_betas}\n\\label{your_table_label}\n\\end{table}"

# open("bbl_large_grid.txt", "w") do file
#     write(file, latex_table)
# end